For the car rental, there are much more states, actions and rewards. 

In [66]:
import math
import time

def poisson(n, lambd):
    return (math.exp(-lambd) * lambd**n) / math.factorial(n)

# We'll use this table to look up probabilities of car rentals and returns.
# This is to optimise learning by avoiding repeated computation. 
poisson_table = dict()
for n, lam in [(i, j) for i in range(21) for j in (2, 3, 4)]:
    poisson_table[(n, lam)] = poisson(n, lam)

In [77]:
states = [(i, j) for i in range(21) for j in range(21)]

# Calculate the expected return of a state given the policy
def state_action_value(action, state, values):
    gamma = 0.9
    if abs(action) > 5:
        print("No more than 5 cars can be moved")
        return 
    # Move cars, clamp to 20 cars
    init_nloc1 = min(state[0] - action, 20)
    init_nloc2 = min(state[1] + action, 20)
    base_reward = -2 * abs(action)
    value = 0
    # We now iterate through every possible combination of returned and requested cars to obtain an expected return. 
    for rented_loc1 in range(init_nloc1 + 1):
        for rented_loc2 in range(init_nloc2 + 1):
            new_init_nloc1 = init_nloc1 - rented_loc1
            new_init_nloc2 = init_nloc2 - rented_loc2
            reward = base_reward + 10 * (rented_loc1 + rented_loc2)
            rented_prob = poisson_table[(rented_loc1, 3)] * poisson_table[(rented_loc2, 4)]
            for returned_loc1 in range(20 - new_init_nloc1 + 1):
                rented_prob_1 = rented_prob * poisson_table[(returned_loc1, 3)]
                for returned_loc2 in range(20 - new_init_nloc2 + 1):
                    nloc1 = new_init_nloc1 + returned_loc1
                    nloc2 = new_init_nloc2 + returned_loc2
                    # Get the probability of this new state occuring
                    prob = rented_prob_1 * poisson_table[(returned_loc2, 2)]
                    # Calculate the return based on the reward and the value of the new state
                    value += prob * (reward + gamma * values[(nloc1, nloc2)])
    return value

def probability_state(rented_loc1, rented_loc2, returned_loc1, returned_loc2):
    return poisson(rented_loc1, 3) * poisson(rented_loc2, 4) * poisson(returned_loc1, 3) * poisson(returned_loc2, 2)

# Policy evaluation function
def evaluate(accuracy, values, policy, iterations = None):
    difference = accuracy
    i = 0
    while difference >= accuracy and (iterations == None or i < iterations):
        difference = 0
        t = time.time()
        for s in states:
            s_value = values[s]
            values[s] = state_action_value(policy[s], s, values)
            difference = max(difference, abs(s_value - values[s]))
        print(f"diff: {round(difference, 4)}, duration: {round(time.time() - t, 4)}")
        i += 1

    return values


In [74]:
def improve(policy, values):
    policy_stable = True
    for s in states:
        old_action = policy[s]
        new_return = 0
        for action in range(-1 * min(5, s[1]), min(5, s[0]) + 1):
            val = state_action_value(action, s, values)
            if val > new_return:
                policy[s] = action
                new_return = val
            
        if old_action != policy[s]:
            policy_stable = False
    return policy_stable, policy

In [75]:
def print_policy(policy):
    for loc1 in range(20, -1, -1):
        for loc2 in range(0, 21):
            print(f'{policy[(loc1, loc2)]}', end = '\t')
        print('\n')

def print_value(values):
    for loc1 in range(20, -1, -1):
        for loc2 in range(0, 21):
            print(f'{round(values[(loc1, loc2)])}', end = '\t')
        print('\n')

In [78]:
policy_stable = False
policy = dict()
value = dict()
for state in states:
    policy[state] = 0
    value[state] = 0

while not policy_stable:
    value = evaluate(0.001, value, policy)
    print_value(value)
    policy_stable, policy = improve(policy, value)
    print('\n')
    print_policy(policy)

diff: 128.6814, duration: 1.9768
diff: 88.9841, duration: 1.8672
diff: 55.0557, duration: 1.8481
diff: 30.8141, duration: 1.8847
diff: 15.7996, duration: 1.8748
diff: 7.537, duration: 1.8819
diff: 3.3714, duration: 1.8644
diff: 1.4376, duration: 1.8801
diff: 0.5861, duration: 1.8981
diff: 0.2314, duration: 1.911
diff: 0.0895, duration: 1.9019
diff: 0.0339, duration: 1.8796
diff: 0.0126, duration: 1.8849
diff: 0.0047, duration: 1.8727
diff: 0.0017, duration: 1.868
diff: 0.0006, duration: 1.8977
1	3	10	22	36	51	65	77	88	97	106	113	120	126	132	137	141	143	143	139	128	

1	4	13	27	45	64	82	98	112	124	136	146	155	164	171	178	183	187	187	181	168	

1	5	15	31	52	74	95	115	132	147	161	174	186	196	206	214	221	225	225	219	203	

1	5	16	34	57	82	105	127	147	165	181	196	210	222	234	243	252	257	257	250	231	

1	5	17	36	60	86	112	136	157	177	195	212	227	241	254	266	275	281	282	274	253	

1	6	17	37	62	90	116	141	164	185	205	223	240	255	269	281	291	298	299	291	269	

1	6	18	38	63	91	119	145	169	191	211	230	